In [17]:
import torch
import torch.nn.functional as F
import torch.nn as nn

MEAN = 0.1307
STANDARD_DEVIATION = 0.3081

In [24]:
# Create Model
class fCNN(nn.Module):
  def __init__(self):
    super(fCNN, self).__init__()

    self.conv1 = nn.Conv2d(1, 32, 3, 1, padding=1)
    self.conv2 = nn.Conv2d(32, 64, 3, 1, padding=1)
    self.drop1 = nn.Dropout(0.25)
    self.drop2 = nn.Dropout(0.5)
    self.fc1 = nn.Linear(12544, 256)
    self.fc2 = nn.Linear(256, 10)

  def forward(self, x):
    # x = x.reshape(84, 84, 4)
    # x = torch.narrow(x, dim=2, start=3, length=1)
    # x = x.reshape(1, 1, 84, 84)
    # x = F.avg_pool2d(x, 3, stride=3)

    out = self.conv1(x)
    out = F.relu(out)
    out = F.max_pool2d(out, 2)
    out = self.conv2(out)
    out = F.relu(out)
    out = self.drop1(out)
    out = torch.flatten(out, 1)
    out = self.fc1(out)
    out = F.relu(out)
    out = self.drop2(out)
    out = self.fc2(out)

    # output = F.softmax(out, dim=1)
    return out

In [25]:
model = fCNN()

In [26]:
model.load_state_dict(torch.load('drive/MyDrive/pytorch/practice2/ffmodel.pt'))

<All keys matched successfully>

In [27]:
model.eval()

fCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (drop1): Dropout(p=0.25, inplace=False)
  (drop2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=12544, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)

In [22]:
dummy_input = torch.zeros(84*84*4)

In [23]:
# Convert to onnx
torch.onnx.export(model, dummy_input, 'drive/MyDrive/pytorch/practice2/ffmnist.onnx', verbose=True)

graph(%0 : Float(28224:1, requires_grad=0, device=cpu),
      %conv1.weight : Float(32:9, 1:9, 3:3, 3:1, requires_grad=1, device=cpu),
      %conv1.bias : Float(32:1, requires_grad=1, device=cpu),
      %conv2.weight : Float(64:288, 32:9, 3:3, 3:1, requires_grad=1, device=cpu),
      %conv2.bias : Float(64:1, requires_grad=1, device=cpu),
      %fc1.weight : Float(256:12544, 12544:1, requires_grad=1, device=cpu),
      %fc1.bias : Float(256:1, requires_grad=1, device=cpu),
      %fc2.weight : Float(10:256, 256:1, requires_grad=1, device=cpu),
      %fc2.bias : Float(10:1, requires_grad=1, device=cpu)):
  %9 : Tensor = onnx::Constant[value= 84  84   4 [ CPULongType{3} ]]()
  %10 : Float(84:336, 84:4, 4:1, requires_grad=0, device=cpu) = onnx::Reshape(%0, %9) # <ipython-input-18-ba257e74f0c7>:14:0
  %11 : Float(84:336, 84:4, 1:1, requires_grad=0, device=cpu) = onnx::Slice[axes=[2], ends=[4], starts=[3]](%10) # <ipython-input-18-ba257e74f0c7>:15:0
  %12 : Tensor = onnx::Constant[value=  1 

In [28]:
# Convert to keras
!pip install pytorch2keras

     |████████████████████████████████| 14.5MB 241kB/s 
  Created wheel for pytorch2keras: filename=pytorch2keras-0.2.4-cp36-none-any.whl size=29663 sha256=c6846299106da65c774220100916bddd26d20211af8bca1f88d25de7666265df
  Stored in directory: /root/.cache/pip/wheels/36/8b/2e/e2b6ae7a78ad4661e6156700bf96816309013f89f7a21f82d7
  Created wheel for onnx2keras: filename=onnx2keras-0.0.24-cp36-none-any.whl size=24580 sha256=6f9c2b8c92550f23b87653e20bdc4340fa3b65cb35f9e881e90107352f6b10c2
  Stored in directory: /root/.cache/pip/wheels/1d/d2/0a/8310ecf5f80355546fadda0fb4c611c8b54a63fce9af555155
Successfully built pytorch2keras onnx2keras


In [29]:
from pytorch2keras.converter import pytorch_to_keras
import numpy as np
from torch.autograd import Variable

In [30]:
input_np = np.random.uniform(0, 1, (1, 1, 28, 28))
input_var = Variable(torch.FloatTensor(input_np))

In [31]:
k_model = pytorch_to_keras(model, input_var)

Unable to use `same` padding. Add ZeroPadding2D layer to fix shapes.


In [32]:
import tensorflow as tf

In [33]:
k_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_0 (InputLayer)         [(None, 1, 28, 28)]       0         
_________________________________________________________________
9_pad (ZeroPadding2D)        (None, 1, 30, 30)         0         
_________________________________________________________________
9 (Conv2D)                   (None, 32, 28, 28)        320       
_________________________________________________________________
10 (Activation)              (None, 32, 28, 28)        0         
_________________________________________________________________
11 (MaxPooling2D)            (None, 32, 14, 14)        0         
_________________________________________________________________
12_pad (ZeroPadding2D)       (None, 32, 16, 16)        0         
_________________________________________________________________
12 (Conv2D)                  (None, 64, 14, 14)        18496 

In [35]:
tf.saved_model.save(k_model, 'drive/MyDrive/pytorch/practice2/k_model')

INFO:tensorflow:Assets written to: drive/MyDrive/pytorch/practice2/k_model/assets


In [36]:
# Convert keras model to tensorflow.js format
!pip install tensorflowjs

     |████████████████████████████████| 71kB 6.2MB/s 
     |████████████████████████████████| 112kB 10.9MB/s 
  Found existing installation: tensorflow-hub 0.11.0
    Uninstalling tensorflow-hub-0.11.0:
      Successfully uninstalled tensorflow-hub-0.11.0


In [37]:
import tensorflowjs as tfjs

In [39]:
tfjs.converters.save_keras_model(k_model, 'drive/MyDrive/pytorch/practice2/tfjsfModel')

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)
